In [45]:
import pyAgrum as gum
import pyAgrum.lib.notebook as gnb

In [46]:
dn = gum.InfluenceDiagram()

In [47]:
# Decision Nodes
call_check_in=dn.addDecisionNode(gum.LabelizedVariable('CallCheckIn', 'call_check_in', ['no', 'yes']))
# Chance Nodes
user_behaviour=dn.addChanceNode(gum.LabelizedVariable('UserBehavior','user_behaviour', ['suspicuous', 'normal']))
account_age=dn.addChanceNode(gum.LabelizedVariable('AccountAge','account_age', ['new', 'old']))
georgraphic_location=dn.addChanceNode(gum.LabelizedVariable('GeographicLocation', 'georgraphic_location', ['rural', 'urban']))
transaction_amount=dn.addChanceNode(gum.LabelizedVariable('TransactionAmount', 'transaction_amount',['0-350000','350001-700000','700000+']))
transaction_info=dn.addChanceNode(gum.LabelizedVariable('TransactionInfo', 'transaction_info', ['abnormal', 'normal']))
transaction_frequency=dn.addChanceNode(gum.LabelizedVariable('TransactionFrequency', 'transaction_frequency', ['frequent', 'infrequent']))
likelihood_of_fraud=dn.addChanceNode(gum.LabelizedVariable('LikelihoodOfFraud', 'likelihood_of_fraud', ['low', 'medium', 'high']))
print ("BN With vars : ",dn)
# Utility Nodes
check_in_needed = dn.addUtilityNode(gum.LabelizedVariable('CallCheckInNeeded', 'check_in_needed', 1))

BN With vars :  Influence Diagram{
  chance: 7,
  utility: 0,
  decision: 1,
  arcs: 0,
  domainSize: 576
}


In [48]:
# Addding arcs between the nodes
arcs = [(user_behaviour, likelihood_of_fraud),(account_age, user_behaviour), (transaction_info, likelihood_of_fraud), (transaction_frequency, transaction_info),(transaction_amount, transaction_info),(georgraphic_location, likelihood_of_fraud)]
for arc in arcs:
    dn.addArc(*arc)
dn

(pyAgrum.InfluenceDiagram<double>@0x7fe9ac26c600) Influence Diagram{
  chance: 7,
  utility: 1,
  decision: 1,
  arcs: 6,
  domainSize: 576
}

In [49]:
# Define the conditional probability tables 

dn.cpt('AccountAge').fillWith([0.35,0.65])
dn.cpt('TransactionFrequency').fillWith([0.92,0.08])
dn.cpt('TransactionAmount').fillWith([0.3,0.4,0.2])

dn.cpt('TransactionInfo')[{'TransactionAmount': 0, 'TransactionFrequency': 0}] = [1, 0]
dn.cpt('TransactionInfo')[{'TransactionAmount': 0, 'TransactionFrequency': 1}] = [0.75, 0.25]
dn.cpt('TransactionInfo')[{'TransactionAmount': 1, 'TransactionFrequency': 0}] = [0.35, 0.65]
dn.cpt('TransactionInfo')[{'TransactionAmount': 1, 'TransactionFrequency': 1}] = [0.175, 0.825]
dn.cpt('TransactionInfo')[{'TransactionAmount': 2, 'TransactionFrequency': 0}] = [0.05, 0.95]
dn.cpt('TransactionInfo')[{'TransactionAmount': 2, 'TransactionFrequency': 1}] = [0, 1]

dn.cpt('GeographicLocation').fillWith([0.33,0.67])

dn.cpt('UserBehavior')[{'AccountAge': 0}] = [0.6, 0.4]
dn.cpt('UserBehavior')[{'AccountAge': 1}] = [0.4, 0.6]


dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 0, 'UserBehavior': 0}] = [0.0, 0.45, 0.55]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 0, 'UserBehavior': 1}] = [0.1, 0.5, 0.4] 
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 1, 'UserBehavior': 0}] = [0.2, 0.5, 0.3]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 0, 'TransactionInfo': 1, 'UserBehavior': 1}] = [0.35, 0.5, 0.15]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 0, 'UserBehavior': 0}] = [0.4, 0.475, 0.125]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 0, 'UserBehavior': 1}] = [0.6, 0.3, 0.1]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 1, 'UserBehavior': 0}] = [0.825, 0.125, 0.05]
dn.cpt('LikelihoodOfFraud')[{'GeographicLocation': 1, 'TransactionInfo': 1, 'UserBehavior': 1}] = [0.915, 0.075, 0.01]

dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 0,'CallCheckIn': 0}] = 20
dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 0, 'CallCheckIn': 1}] = 12
dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 1, 'CallCheckIn': 0}] = 0
dn.utility('CallCheckInNeeded')[{'LikelihoodOfFraud': 1, 'CallCheckIn': 1}] = -2


In [50]:
dn

(pyAgrum.InfluenceDiagram<double>@0x7fe9ac26c600) Influence Diagram{
  chance: 7,
  utility: 1,
  decision: 1,
  arcs: 6,
  domainSize: 576
}

In [51]:
# With no evidence
# gnb.showInference(dn, evs={})
# dn.saveBIFXML("DN_scenario0.bif")

In [52]:
#setting evidence
#scenario 1 (conditions applicable for workshop)
# gnb.showInference(dn, evs={'TransactionAmount':'0-350000', 'TransactionFrequency':'infrequent', 'AccountAge':'old'})
# dn.saveBIFXML("DN_scenario1.bif")

In [53]:
#scenario2 (evidence not applicable for workshop)
# gnb.showInference(dn, evs={'FamilySupport':'yes', 'Attendance':'high','Internet': 'yes','FamilyIncome':'700000+'})
# dn.saveBIFXML("DN_scenario2.bif")

In [54]:
#scenario3 (evidence for an ambiguous decision)
# gnb.showInference(dn, evs={'FamilySupport':'no', 'Attendance':'high','Internet': 'yes','FamilyIncome':'350001-700000'})
# dn.saveBIFXML("DN_scenario3.bif")